In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Adaboost/

/content/gdrive/MyDrive/Adaboost


In [3]:
!pip install numpy scikit-learn pandas matplotlib

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import minmax_scale, StandardScaler, MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

np.random.seed(16450)

In [6]:
def normalize(dataset):
  for i in dataset.columns:
    if(dataset[i].nunique()>2 and dataset[i].dtype != 'object'):
      dataset[i] = (dataset[i]-dataset[i].min())/(dataset[i].max()-dataset[i].min())
  return dataset

In [7]:
# label = the name of prediction column
label = ''

In [8]:
def preprocess1():
  dataset = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

  dataset.drop(['customerID'] , inplace=True , axis='columns')

  for i in dataset.columns:
    #print(i, dataset[i].nunique())
    if(dataset[i].nunique() == 2 and dataset[i].dtype == 'object'):
      #print(dataset[i].unique()[0], dataset[i].unique()[1])
      dataset[i] = dataset[i].map({dataset[i].unique()[0] : 0, dataset[i].unique()[1] : 1})

  dataset['TotalCharges'] = pd.to_numeric(dataset['TotalCharges'], errors='coerce')

  # mean = dataset['TotalCharges'].mean()
  # dataset['TotalCharges'] = dataset['TotalCharges'].replace([np.nan],mean)
  arr = []
  for i in range(len(dataset['tenure'])):
    if(dataset.loc[i , 'tenure'] == 0):
      imagined_tenure = random.random()
      # print(imagined_tenure)
      # print(dataset.loc[i , ['MonthlyCharges']])
      the_loc = dataset.loc[i , ['MonthlyCharges']]
      # print(the_loc)
      dataset.at[i , ['TotalCharges']] = float(the_loc * imagined_tenure)
      # print(dataset.loc[i , ['TotalCharges']])
      arr.append(i)
  #dataset.info()
  # for i in dataset['TotalCharges']:
  #   if(np.isnan(i)):
  #     i.replace(mean)

  dataset['Churn'] = dataset['Churn'].map({1: 1, 0: -1})
  print(dataset['Churn'].unique())


  dataset = pd.get_dummies(dataset)
  #print(dataset.head())

  global label
  label = 'Churn'

  return dataset




#dataset = normalize(dataset)



#dataset = dataset[1000:3050]
# dataset.head(100)




In [9]:
def preprocess2(file_name):
  dataset = pd.read_csv(file_name)

  global label

  label = ' Income'

  for i in dataset.columns:
    max_frequent_value = dataset[i].value_counts().idxmax()
    dataset[i] = dataset[i].replace(' ?', max_frequent_value)


  if(file_name == 'adult_test.csv'):
    dataset[label] = dataset[label].map({' <=50K.': (-1), ' >50K.': (1)})

  else:
    dataset[label] = dataset[label].map({' <=50K': (-1), ' >50K': (1)})



  for i in dataset.columns:
    if(dataset[i].nunique() == 2 and dataset[i].dtype == 'object'):
      #print(dataset[i].unique()[0], dataset[i].unique()[1])
      dataset[i] = dataset[i].map({dataset[i].unique()[0] : 0, dataset[i].unique()[1] : 1})
  
  

  #dataset.info()

  dataset = pd.get_dummies(dataset)



  # print(dataset.count().unique())

  

  return dataset





In [10]:
def preprocess3():
  global label
  label = 'Class'

  dataset = pd.read_csv('creditcard.csv')

  dataset['Class'] = dataset['Class'].map({1: 1, 0: -1})
  # print(dataset.isnull().sum())

  df = dataset[dataset['Class'] == 1]

  class_1_index = df.index

  dataset.drop(class_1_index)

  print(df.shape)

  sampled = dataset.sample(n=5000)

  print(sampled.shape)

  df2 = pd.concat([df, sampled])

  df2 = shuffle(df2)

  # print(df)

  return df2

dataset = preprocess3()

# print(dataset.head(100))

# print(dataset.tail(100))

dataset.shape

print(np.any(np.isnan(dataset.to_numpy())))

# dataset

(492, 31)
(5000, 31)
False


In [11]:
def preprocess_online():
  dataset = pd.read_csv('dataset.txt')

  dataset = dataset.drop(['N'], axis=1)

  dataset.info()

  global label
  label = 'Label'

  for i in dataset.columns:
    max_frequent_value = dataset[i].value_counts().idxmax()
    dataset[i] = dataset[i].replace(' ?', max_frequent_value)

  dataset[label] = dataset[label].map({' <=50K': (-1), ' >50K': (1)})

  for i in dataset.columns:
    if(dataset[i].nunique() == 2 and dataset[i].dtype == 'object'):
      #print(dataset[i].unique()[0], dataset[i].unique()[1])
      dataset[i] = dataset[i].map({dataset[i].unique()[0] : 0, dataset[i].unique()[1] : 1})

  

  dataset = pd.get_dummies(dataset)

  return dataset 

preprocess_online()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       32561 non-null  int64 
 1   B       32561 non-null  object
 2   C       32561 non-null  int64 
 3   D       32561 non-null  object
 4   E       32561 non-null  int64 
 5   F       32561 non-null  object
 6   G       32561 non-null  object
 7   H       32561 non-null  object
 8   I       32561 non-null  object
 9   J       32561 non-null  object
 10  K       32561 non-null  int64 
 11  L       32561 non-null  int64 
 12  M       32561 non-null  int64 
 13  Label   32561 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.5+ MB


,A,C,E,J,K,L,M,Label,B_ Federal-gov,B_ Local-gov,B_ Never-worked,B_ Private,B_ Self-emp-inc,B_ Self-emp-not-inc,B_ State-gov,B_ Without-pay,D_ 10th,D_ 11th,D_ 12th,D_ 1st-4th,D_ 5th-6th,D_ 7th-8th,D_ 9th,D_ Assoc-acdm,D_ Assoc-voc,D_ Bachelors,D_ Doctorate,D_ HS-grad,D_ Masters,D_ Preschool,D_ Prof-school,D_ Some-college,F_ Divorced,F_ Married-AF-spouse,F_ Married-civ-spouse,F_ Married-spouse-absent,F_ Never-married,F_ Separated,F_ Widowed,G_ Adm-clerical,G_ Armed-Forces,G_ Craft-repair,G_ Exec-managerial,G_ Farming-fishing,G_ Handlers-cleaners,G_ Machine-op-inspct,G_ Other-service,G_ Priv-house-serv,G_ Prof-specialty,G_ Protective-serv,G_ Sales,G_ Tech-support,G_ Transport-moving,H_ Husband,H_ Not-in-family,H_ Other-relative,H_ Own-child,H_ Unmarried,H_ Wife,I_ Amer-Indian-Eskimo,I_ Asian-Pac-Islander,I_ Black,I_ Other,I_ White
0,39,77516,13,0,2174,0,40,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,50,83311,13,0,0,0,13,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,38,215646,9,0,0,0,40,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,53,234721,7,0,0,0,40,-1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,1,0,0,40,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,1,0,0,38,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
32557,40,154374,9,0,0,0,40,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
32558,58,151910,9,1,0,0,40,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
32559,22,201490,9,0,0,0,20,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [12]:
# dataset 1 start

# dataset = preprocess1()
# x = dataset.drop([label], axis='columns')
# y = dataset[label]

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# dataset 1 end


# dataset 2 start

# data_train = preprocess2('adult.csv')
# data_test = preprocess2('adult_test.csv')

# missing_cols = set( data_train.columns ) - set( data_test.columns )
# for c in missing_cols:
#   data_test[c] = 0
# data_test = data_test[data_train.columns]
  


# x_train = data_train.drop([label], axis='columns')
# y_train = data_train[label]



# x_test = data_test.drop([label], axis='columns')
# y_test = data_test[label]


# dataset  2 end

# dataset 3 start

# dataset = preprocess3()

# x = dataset.drop([label], axis='columns')
# y = dataset[label]


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# dataset 3 end

# online dataset

dataset = preprocess_online()

x = dataset.drop([label], axis='columns')
y = dataset[label]


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)


# online dataset end



# common for all

x_train = normalize(x_train)
x_test = normalize(x_test)


x_train_matrix = x_train.values
x_test_matrix = x_test.values
y_train_matrix = y_train.values.reshape(-1,1)
y_test_matrix = y_test.values.reshape(-1,1)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       32561 non-null  int64 
 1   B       32561 non-null  object
 2   C       32561 non-null  int64 
 3   D       32561 non-null  object
 4   E       32561 non-null  int64 
 5   F       32561 non-null  object
 6   G       32561 non-null  object
 7   H       32561 non-null  object
 8   I       32561 non-null  object
 9   J       32561 non-null  object
 10  K       32561 non-null  int64 
 11  L       32561 non-null  int64 
 12  M       32561 non-null  int64 
 13  Label   32561 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.5+ MB


In [13]:
# iterative approach, takes too long


def cost_function_derivativetanh(x_train,y_train,theta,j,m,alpha):
    sumErrors = 0
    for i in range(m):
        xi = x_train.iloc[i]
        hi = hypothesistanh(theta,xi)
        yt = y_train.iloc[i]
        hit = np.subtract((float)(hi),float(yt))
        error = np.dot(hit,x_train.iloc[i][j])
        error = np.dot(error,(1-hi*hi))
        sumErrors += error
    #m = len(y_test)
    constant = float(alpha)/float(m)
    j = constant * sumErrors
    #print("SumErrors", sumErrors)
    return j

def gradient_descenttanh(x_train,y_train,theta,m,alpha):
    new_theta = []
    for j in range(len(theta)):
        new_theta_value = theta[j] - cost_function_derivativetanh(x_train,y_train,theta,j,m,alpha)
        new_theta.append(new_theta_value)
    #print ('theta ', new_theta)	
    return new_theta


def error_calc(x_train,y_train,theta):
  total = 0
  m = len(y_train)
  for i in range(m):
        xi = x_train.iloc[i]
        hi = hypothesistanh(theta,xi)
        yt = y_train.iloc[i]
        hit = np.subtract((float)(hi),float(yt))
        total += hit*hit

  return total/float(m)

def logistic_regression_tanh(x_train,y_train,alpha,theta,num_iters, threshold):
    m = len(y_train)
    for x in range(num_iters):
        new_theta = gradient_descenttanh(x_train,y_train,theta,m,alpha)
        theta = new_theta
        total = error_calc(x_train,y_train,theta)
        if(total < threshold):
          break
        #if x % 1000 == 0:
            #cost_functiontanh(theta,m)
        #print ('theta ', theta)
        #print('total' , total)	
            #print ('cost is ', cost_functiontanh(theta,m))
    score = 0
    length = len(x_test)
    for i in range(length):
        prediction = hypothesistanh(theta,x_test.iloc[i])
        
        if prediction < 0:
          prediction = -1
        else:
          prediction = 1
        # if(prediction == -1):
        #   print("prediction ", prediction)
        answer = y_test.iloc[i]
        if prediction == answer:
            score += 1
    my_score = float(score) / float(length)
    print ('Your score with tanh: ', my_score)

    return theta

In [14]:
def Tanh(z):
    th = np.tanh(z)
    # if(np.isnan(th)):
    #   if(z < 0):
    #     return -1
    #   else:
    #     return 1

    return th

In [15]:
def hypothesistanh(theta,x):
    #print ('theta ', theta)	
    # z = 0
    # for i in range(len(theta)):
    #     xi = x[i]
    #     z += np.dot(xi,theta[i].transpose())

    z = np.dot(x,theta)
    # z = x @ theta
    # print('z.shape',z.shape)
    return Tanh(z)

In [16]:
def mean_error(y , y_hat):
  diff = y - y_hat
  return np.mean(np.power(diff , 2))

In [17]:
def gradient_descent(X, y, y_hat, lr):
    m = X.shape[0]
    
    diff = y - y_hat
    derivative = 1 - np.power(y_hat , 2)
    multi = diff * derivative

    dw = np.dot(X.T , multi)
    
    constant = (1/float(m)) * float(lr) 
    return constant * dw

In [18]:
def log_reg(X, y, iterations, lr, threshold):
  m,n = X.shape

  w = np.zeros((n+1 , 1))
  # print(w.shape)

  X_train = np.concatenate((np.ones((m, 1)), X), axis=1)


  errors = []

  for i in range(iterations):
    y_hat = hypothesistanh(w , X_train)

    decay = gradient_descent(X_train , y , y_hat , lr)

    if(np.isnan(decay.any())):
      print('decay')
    
    w = w + decay
    

    error = mean_error(y , y_hat)
    errors.append(error)

    if error < threshold:
      print('bingo', i)
      break

  score = 0
  length = x_test_matrix.shape[0]
  X_test = np.concatenate((np.ones((length, 1)), x_test_matrix), axis=1)
  
  
  prediction = hypothesistanh(w, X_test)
  
  # for i in range(length):
  #   if prediction[i][0] < 0:
  #     prediction[i][0] = -1
  #   else:
  #     prediction[i][0] = 1
    
  #   answer = y_test_matrix[i][0]
  #   if prediction[i][0] == answer:
  #     score += 1

  # # print('pred uniq ', np.unique(prediction, axis=0))
  # my_score = float(score) / float(length)
  # print ('Your score with tanh: ', my_score)
  
  return w


In [19]:
def test_logistic(x, y):
  alpha = 0.1
  iterations = 100
  threshold = 0.5
  w = log_reg(x_train_matrix , y_train_matrix , iterations , alpha , threshold)
  length = x.shape[0]
  X_test = np.concatenate((np.ones((length, 1)), x), axis=1)
  
  
  prediction = hypothesistanh(w, X_test)

  for i in range(length):
    if prediction[i][0] < 0:
      prediction[i][0] = -1
    else:
      prediction[i][0] = 1

  matrix = confusion_matrix(y , prediction)

  TN = matrix[0][0]
  FP = matrix[0][1]
  FN = matrix[1][0]
  TP = matrix[1][1]

  print('accuracy :' , (TP+TN)/(TP+TN+FP+FN))
  recall = TP/(TP+FN)
  print('sensitivity/recall :' , recall)
  specificity = TN/(TN+FP)
  print('specificity :' , specificity)
  precision = TP/(TP+FP)
  print('precision/PPV :' , precision)
  false_discovery_rate = 1 - precision
  print('false discovery rate :' , false_discovery_rate)
  f1_score = 2 * ((recall*precision) / (recall + precision))
  print('F1 score :' , f1_score)




In [20]:
# alpha = 0.1
# iterations = 100
# threshold = 0.5
# w = log_reg(x_train_matrix , y_train_matrix , iterations , alpha , threshold)

def Test_logistic():
  print('training set')
  test_logistic(x_train_matrix, y_train_matrix)
  print()
  print('test set')
  test_logistic(x_test_matrix, y_test_matrix)

Test_logistic()

training set
accuracy : 0.8241536991536992
sensitivity/recall : 0.46255694760820043
specificity : 0.9381508078994614
precision/PPV : 0.7021828398530365
false discovery rate : 0.29781716014696347
F1 score : 0.557720367350442

test set
accuracy : 0.8071845256370893
sensitivity/recall : 0.4369645042839657
specificity : 0.9311475409836065
precision/PPV : 0.68
false discovery rate : 0.31999999999999995
F1 score : 0.5320417287630402


In [21]:
# a = np.array([[1,2],[2,3]])
# b = np.array([4,5])
# a.shape
# c = a @ b
# np.concatenate((np.ones((x_train_matrix.shape[0], 1)),x_train_matrix), axis=1)

In [22]:
def ada_boost(num_iter):
  m = y_train_matrix.shape[0]

  weight = np.empty(m)
  weight.fill(1.0/float(m))
  #print(weight)
  
  hypothesis_container = []
  hypothesis_weight = []

  

  x_train_sampled = np.copy(x_train_matrix)
  
  y_train_sampled = np.copy(y_train_matrix)

  for i in range(num_iter):
    
    resampled_int = np.random.choice(x_train_matrix.shape[0], size=m, p=weight)
    
    x_train_sampled = x_train_matrix[resampled_int , :]
    y_train_sampled = y_train_matrix[resampled_int , :]
    
    
    theta = log_reg(np.copy(x_train_sampled), y_train_sampled, iterations , alpha, 0)
    

    error = 0

    x_train_sampled_2 = np.concatenate((np.ones((m, 1)), x_train_sampled), axis=1)
    #print(x_train_sampled_2)

    h = hypothesistanh(theta , x_train_sampled_2)
    #print('go4')
    for j in range(m):
      #xj = x_train_sampled.iloc[j]
      #h = hypothesistanh(theta,xj)
      if h[j][0] < 0:
        h[j][0] = -1
      else:
        h[j][0] = 1
      yj = y_train_sampled[j][0]
      if h[j][0] != yj:
        error = error + weight[j]

    if error > 0.5:
      #print(error)
      continue

    h = hypothesistanh(theta , x_train_sampled_2)
    for j in range(m):
      # xj = x_train_sampled.iloc[j]
      # hj = hypothesistanh(theta,xj)
      if h[j][0] < 0:
        h[j][0] = -1
      else:
        h[j][0] = 1
      yj = y_train_sampled[j][0]
      if h[j][0] == yj:
        weight[j] = weight[j] * float(error)/(1.0-float(error))

    weight = weight/np.sum(weight)
    #print(weight)

    hypothesis_container.append(theta)
    hypothesis_weight.append(np.log2(  (1.0-float(error))   /   float(error)   ))

  
  X_test = np.concatenate((np.ones((x_test_matrix.shape[0], 1)), x_test_matrix), axis=1)
  
  #print('go')

  h_cap = Weighted_majority(X_test , hypothesis_container , hypothesis_weight)

  X_train = np.concatenate((np.ones((x_train_matrix.shape[0], 1)), x_train_matrix), axis=1)

  h_cap2 = Weighted_majority(X_train , hypothesis_container , hypothesis_weight)
  #print(h_cap)
  #print('go2')

  m2 = len(x_train)
  score = 0
  for i in range(m2):
    if h_cap2[i] < 0:
      h_cap2[i] = -1
    else:
      h_cap2[i] = 1

    answer = y_train_matrix[i][0]
    if h_cap2[i] == answer:
      score += 1
  # print(h_cap)
  # print(np.unique(h_cap))
  print('adaboost on train ' ,float(score) / float(m2))

  m2 = len(x_test)
  score = 0
  for i in range(m2):
    if h_cap[i] < 0:
      h_cap[i] = -1
    else:
      h_cap[i] = 1

    answer = y_test_matrix[i][0]
    if h_cap[i] == answer:
      score += 1
  # print(h_cap)
  # print(np.unique(h_cap))
  print('adaboost on test ' ,float(score) / float(m2))

  





def Weighted_majority(x , hypothesis_container , hypothesis_weight):
  m = len(x)
  iter = len(hypothesis_weight)
  #print(iter)
  h_cap = np.zeros(m)
  for k in range(iter):
    h = hypothesistanh(hypothesis_container[k].reshape(-1, 1), x)
    for i in range(m):
      
      mul = h[i][0]*hypothesis_weight[k]
      h_cap[i] = h_cap[i] + mul

  #print(h_cap)
  return h_cap



  








In [23]:
x_test.shape

(3257, 63)

In [24]:
initial_theta = np.zeros(x_test.iloc[1].shape)
#x_test.iloc[1]
alpha = 0.1
iterations = 1000
#logistic_regression_tanh(alpha,initial_theta,iterations)
i = 5
for _ in range(4):
  ada_boost(i)
  i+=5


adaboost on train  0.8345277095277095
adaboost on test  0.8219220141234265
adaboost on train  0.8338452088452089
adaboost on test  0.8182376420018422
adaboost on train  0.8317635817635818
adaboost on test  0.8191587350322382
adaboost on train  0.8267472017472017
adaboost on test  0.8096407737181456
